In [1]:
# for i in range(4):
#     %run log_analyzer.ipynb

In [ ]:
# Suppress cargo build warnings
%env RUSTFLAGS=-Awarnings
for i in range(10):
    print("Running CRF", i)
    ! cargo run --bin evaluate_feature_detection_transcode --release --features "open-cv feature-logging" -- \
            --crf {i} --delta-t-max 76500 --frame-count-max 500 \
            --input-filename "/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_040103_07_001011_001093.mp4" \
            --scale 1.0 --detect-features \
            --color-input


env: RUSTFLAGS=-Awarnings
Running CRF 0
   Compiling opencv v0.83.0
   Compiling zstd v0.12.4
   Compiling aedat v1.3.3=========> ] 332/340: zstd, opencv(build)         
